# Here I will test how my k-nn model works

I'll be trying it with the iris dataset to see if it works, so first we import all the tools needed

In [1]:
from knn import KNN

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

iris = load_iris()
X = iris.data # Features
Y = iris.target # Tags

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, train_size=0.7, random_state=42)

Now we have the data set ready to test the knn implementation

In [2]:
knn = KNN(k=5)
knn.fit(X_train, Y_train)

#Y_predict = [knn.predict(x) for x in X_test]

Found the error, it seems that the distances evaluate to None. This was the code which fails:

```def gather_distances(self, x_0, d_formula=euclidean_distance):
        distances = []
        for x in self.X_train:
            distances.append(d_formula(x_0, x))
        
        return distances```
The problem was in the distance formula, I FORGOT THE RETURN -_-

In [3]:
neighbours = knn.gather_distances(X_test[0])
print(neighbours)

[np.float64(1.2489995996796794), np.float64(0.5385164807134502), np.float64(1.0862780491200217), np.float64(0.648074069840786), np.float64(2.345207879911715), np.float64(0.6557438524301997), np.float64(2.5903667693977237), np.float64(0.6480740698407862), np.float64(3.6578682316343767), np.float64(2.683281572999748), np.float64(0.7071067811865478), np.float64(3.6138621999185307), np.float64(3.7509998667022106), np.float64(3.2372828112477294), np.float64(1.8248287590894656), np.float64(0.670820393249937), np.float64(3.3000000000000003), np.float64(3.443835071544513), np.float64(3.277193921634788), np.float64(0.7211102550927979), np.float64(3.5496478698597698), np.float64(0.6708203932499366), np.float64(2.925747767665559), np.float64(3.7815340802378077), np.float64(0.43588989435406783), np.float64(0.9899494936611662), np.float64(3.769615364994153), np.float64(0.6480740698407861), np.float64(0.8660254037844383), np.float64(0.9486832980505138), np.float64(0.8124038404635955), np.float64(1.4

Now we have the distances, so I proceed with the next step

In [4]:
neighbours = knn.get_neighbours(neighbours)
print(neighbours)


[1 1 1 1 1]


def most_common(self, neighbours): # Receives a np.array
        k_neighbours = Counter(neighbours) 

        most_common, most_common_count = k_neighbours(1)[0]
        #I want to know how many neighbours share the top
        top_commons = len(count for count in k_neighbours.values() if count == most_common_count)
        if top_commons == 1:
            return most_common # If there is one top neighbours we return it
        else:
            return most_common(k_neighbours[:-1]) # Otherwise we slice the last neighbour and count again



This output error code

TypeError 
Cell In[5], line 1
----> 1 classify = knn.most_common(neighbours)
      2 print(classify)

File ~/projects/models_from_scratch/knn/knn.py:47, in KNN.most_common(self, neighbours)
     44 def most_common(self, neighbours): # Receives a np.array
     45     k_neighbours = Counter(neighbours) 
---> 47     most_common, most_common_count = k_neighbours(1)[0]
     48     #I want to know how many neighbours share the top
     49     top_commons = len(count for count in k_neighbours.values() if count == most_common_count)

TypeError: 'Counter' object is not callable

In [5]:
classify = knn.most_common(neighbours)
print(classify)

1


Now it works! This was the bugged line, where I was incorrectly invoking the Counter when I needed to call most_common.Counter:

`most_common, most_common_count = k_neighbours(1)[0] # I was calling the Counter and not most_common.Counter`

Finally it seems that the model performs correctly.

To see if it really works well I will compare it with the scikit-learn knn model. So first we import the model and train it

In [6]:
from sklearn.neighbors import KNeighborsClassifier

knn_sklearn = KNeighborsClassifier(3)
knn_sklearn.fit(X_train, Y_train)

KNeighborsClassifier(n_neighbors=3)

Now mine!

In [7]:
knn_from_scratch = KNN(k=3)
knn_from_scratch.fit(X_train, Y_train)

Now, to the predictions

In [8]:
Y_predict_sklearn = knn_sklearn.predict(X_test)

Y_predict_from_scratch = [knn_from_scratch.predict(x) for x in X_test]

In [9]:
from sklearn.metrics import accuracy_score

sklearn_accuracy = accuracy_score(Y_test, Y_predict_sklearn)

from_scratch_accuracy = accuracy_score(Y_test, Y_predict_from_scratch)

print(f"Precision of KNN sklearn model: {sklearn_accuracy}\n")

print(f"Precision of KNN custom model: {from_scratch_accuracy}")

Precision of KNN sklearn model: 1.0

Precision of KNN custom model: 1.0


Sadly it performs poorly :( , but it performs at least :) 
Finally I found a bug on how I was calling get_neighbours. Because I separated gather_distances from it now I needed to pass the distances as an argument, and not x_0. Works fine now.

Before

``` def predict(self, x_0):
        predicted_class = self.most_common(self.get_neighbours(x_0))
        return predicted_class ```

Corrected

``` def predict(self, x_0):
        d_from_x = self.gather_distances(x_0)
        predicted_class = self.most_common(self.get_neighbours(d_from_x))
        return predicted_class ```